참고: https://lwoongh38.github.io/bert-sentiment-keyword  
https://github.com/lwoongh38/project_cs1/blob/main/keyword_extractor_bert.ipynb

In [33]:
import pandas as pd
#import seaborn as sns
import numpy as np
# import pykospacing
# spacer = pykospacing.Spacing()

import re
#import matplotlib.pyplot as plt
#%matplotlib inline
from collections import Counter
import time
#from sklearn.model_selection import train_test_split
#import networkx as nx
#import matplotlib.pyplot as plt
from pykospacing import Spacing

### 총 데이터 합치기
키워드 추출에서 깃 데이터는 제외 (상품명이 없기 때문에)

In [34]:
df_trn = pd.read_csv('data/train.csv')
df_tst = pd.read_csv('data/test.csv')
df_craw = pd.read_csv('data/Crawling_final.csv')

In [35]:
df_hub = pd.concat([df_trn, df_tst], ignore_index=True)

### 클래스 분류

In [36]:
# 1~100점으로 분포된 평점에서 1~50은 0(부정), 51~100은 1(긍정)
df_hub['labels'] = df_hub['ReviewScore'].apply(lambda x: 0 if 0 <= x <= 50 else 1)

# 1~5점으로 분포된 평점에서 1~2은 0(부정), 4~5은 1(긍정)
df_craw['labels'] = df_craw['ReviewScore'].apply(lambda x: 0 if 1 <= x <= 2 else 1)

In [37]:
# 데이터프레임을 리스트에 담기
dfs = [df_hub, df_craw]

# 데이터프레임 위아래로 합치기
df = pd.concat(dfs, ignore_index=True)

In [38]:
# RawText 열에서 중복인 내용이 있다면 중복 제거
df.drop_duplicates(subset=['RawText'], inplace=True) # RawText 열에서 중복인 내용이 있다면 중복 제거
df.drop(columns=['Index', 'Unnamed: 0'], inplace=True)
df.dropna(subset=['RawText', 'ProductName'], inplace=True)
df.reset_index(drop=True, inplace=True) # 인덱스 재설정
print('총 샘플의 수 :',len(df))

총 샘플의 수 : 512889


In [39]:
df

,RawText,ReviewScore,ProductName,labels
0,머리결이 푸석해졌어요. 염색시간은 3시간정도 해야하는데 너무 번거로워요.,5,OO 헤나 파우더 100g 1개,0
1,천연헤나가 좋다는 건 알지만 흰머리 염색은 너무 번거롭고 오래걸리는 단점이 있네요....,5,OO 헤나 파우더 100g 1개,0
2,"다크 브라운으로 염색했는데, 발색이 블랙처럼 너무 어둡습니다. 염색 다음날 투명으로...",5,OO 헤나 파우더 100g 1개,0
3,염색할때 헤나를 계속 사용했는데요 ㅇㅇㅇ는 처음 사용해보네요. 염색은 잘 되는것 같...,4,OO 헤나 파우더 100g 1개,0
4,"첫번째 주문입니다. 저렴한 가격도 아닌데, 염색이 잘 되지 않습니다. 사용하기도 불...",5,OO 헤나 파우더 100g 1개,0
...,...,...,...,...
512884,매번 주문해요!! 좋아요~!,5,만년도장 아크릴4푼검정,1
512885,좋아요 매번 주문해서 사용중~~~~~,5,만년도장 아크릴4푼검정,1
512886,ㅋㅋ 주문을 잘못했어요 한글 두개로 ㅋㅋ 하나는 한자로 해야되는데 졸지에 명인됨 ㅋㅋ,5,만년도장 아크릴4푼검정,1
512887,도장찍을때마다 인주가 필요없어 넘 편해요,5,만년도장 아크릴4푼검정,1


### 정제

In [40]:
df.ProductName.nunique()

10134

In [41]:
df = df.groupby(['ProductName', 'labels'])['RawText'].apply(lambda x: ' '.join(x)).reset_index()
df = df.sort_values(by='ProductName', ascending=True)

In [42]:
df

,ProductName,labels,RawText
0,LG 울트라HD TV 75형(189cm) 75UN7850KNA+LG사운드바,1,화면이 커서 너무 좋아요 기사분도 친절하시고 배송 날짜보다 빨리 배송되어 잘 보고 ...
1,[22년 신모델] LG그램 15Z95P-GA5LK (i5-1155G7/16GB/...,1,"입학축하선물로 노트북 뭘 살지 이것저것 다 찾아보다가, 결국 OOOO 22형으로 주..."
2,무선키보드 마우스 세트 저소음 옵티컬 4버튼 해킹방지 ONG ENTUS OGN-W...,1,배송은 바르고 좋은데 저소음이라고 샀는데 기대이상이네요 그런데 마우스가 좀 빡빡한...
3,(1+1 TJ 태진 블루투스 마이크 / 무선 노래방 마이크,1,코로나로 노래방을 못가는 아이들이 어디서나 노래부를 수 있어 완전 대~~~박! 시험...
4,(1+1) OO 휴대용 마스크 살균건조기,1,오늘받았는데 어머나 너무좋아요 마스크바로넣어 살균 불깜박거리고 너무좋은데요 마스크...
...,...,...,...
12266,휴대용 다용도 옷걸이 건조기 제습기 HG-660DY,1,"간편한 옷걸이 건조기, 휴대하기도 좋고 찜찜한 옷 건조하기도 좋아 만족해요"
12267,휴대폰 태블릿 거치대 자바라 130cm,1,정면 카메라 말고 카메라가 하나 더 필요해서 핸드폰 거치대를 이용해 보려고 주문했...
12268,휴먼케어 감면 하이패스 시거잭전원타입 HUMC-200,0,장애인 혜택을 못받아서 좀 아쉽지만 가성비 갑인 제품으로 추천해요 다만 지문인식 기...
12269,휴플러스 어깨안마기 HPR-190,1,가격이 저렴해서 구매했는데 전에 쓰던거랑 볼의 크기가 좀 작네요 그냥저냥 쓸만해요....


In [43]:
df.loc[df['ProductName'] == '오* 무드1000 미니 가습기']

,ProductName,labels,RawText
11590,오* 무드1000 미니 가습기,0,잘사용중이였는데 갑자기 가습방향이 아래로만 내려가서 ... 이상하네요?\n수건깔아도...
11591,오* 무드1000 미니 가습기,1,소리는 조금 나요.\n물은 750ml정도 넣어야 잘나와요..\n다른건 괜찮은거 같아...


In [44]:
# 줄바꿈 문자 제거
df['RawText'] = df['RawText'].str.replace('\n', " ", regex=False)

# 한글과 공백을 제외하고 모두 제거
df['RawText'] = df['RawText'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", " ", regex=True)

# 띄어쓰기 교정
spacing = Spacing()
df['RawText'] = df['RawText'].apply(spacing)

In [45]:
df.loc[df['ProductName'] == '오* 무드1000 미니 가습기']

,ProductName,labels,RawText
11590,오* 무드1000 미니 가습기,0,잘 사용 중이였는데 갑자기 가습 방향이 아래로만 내려가서 이상하네요 수건 깔아도 다...
11591,오* 무드1000 미니 가습기,1,소리는 조금 나요 물은 정도 넣어야 잘 나와 요 다른 건 괜찮은 거 같아 요 이 거...


In [46]:
df.to_csv('data/keyword_preprocessed_data.csv', index=False) # 혹시 몰라서 저장